## 🚀 Model Deployment

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sqlite3
from glob import glob
import joblib
import pandas as pd
import requests
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from config import settings
from data import SQLRepository
from model import GarchModel


In [2]:
# Initialize repository
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)
print("repo type :",type(repo))
print("repo.connection type :" , type(repo.connection))

repo type : <class 'data.SQLRepository'>
repo.connection type : <class 'sqlite3.Connection'>


In [3]:
#Instantiate a 'GarchModel'
gm_amazon = GarchModel(ticker="AMZN",repo=repo,use_new_data=False)

In [4]:
#Does 'gm_amazon' have the correct attributes?
assert gm_amazon.ticker == "AMZN"
assert gm_amazon.repo == repo
assert not gm_amazon.use_new_data
assert gm_amazon.model_directory == settings.model_directory
print("GarchModel attributes verified.")


GarchModel attributes verified.


After working on Garch model wrangle_data method next

In [5]:
#Instantiate a 'GarchModel'
model_netflix= GarchModel(ticker="NFLX",repo=repo,use_new_data=True)
#Check that model doesn't have 'data' attribute
assert not hasattr(model_netflix,"data")

In [6]:
#Wrangle data
model_netflix.wrangle_data(n_observations=2500)

date
2015-06-17   -0.010511
2015-06-18    0.005001
2015-06-19   -0.009198
2015-06-22    0.027089
2015-06-23    0.009320
                ...   
2025-05-19    0.000092
2025-05-20    0.000319
2025-05-21    0.002190
2025-05-22   -0.005575
2025-05-23   -0.002172
Name: return, Length: 2500, dtype: float64

In [7]:
#Does model now have "data" attribute?
assert hasattr (model_netflix,"data")
#Is the 'data' a Series?
assert isinstance(model_netflix.data,pd.Series)
#Is series correct shape?
assert model_netflix.data.shape == (2500,)

#### GarchModel Class : Fit method

In [8]:
# GarchModel Class: Fit method
from arch.univariate.base import ARCHModelResult  


# Instantiate GarchModel using old data
model_netflix = GarchModel(ticker="NFLX", repo=repo, use_new_data=False)

# Wrangle data
model_netflix.wrangle_data(n_observations=2500)

# Fit GARCH(1,1) model to data
model_netflix.fit(p=1, q=1)

# Does model_netflix have a 'model' attribute?
assert hasattr(model_netflix, "model")

# Is model the correct data type?
assert isinstance(model_netflix.model, ARCHModelResult)

# Does model have correct parameters?
assert model_netflix.model.params.index.to_list() == ["mu", "omega", "alpha[1]", "beta[1]"]

print("fit tests passed.")
#Check model parameters
model_netflix.model.summary()

fit tests passed.


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                5397.07
Distribution:                  Normal   AIC:                          -10786.1
Method:            Maximum Likelihood   BIC:                          -10762.8
                                        No. Observations:                 2500
Date:                Sat, May 24 2025   Df Residuals:                     2499
Time:                        12:40:05   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         2.0727e-03  1.786e-03      1.161      0.246 [-1.427e-03,5.573e-03]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      2.0831e-05  7.320e-06      2.846  4.430e-03 [6.484e-06,3.518e-05]
alpha[1]       0.0500      0.211      0.237      0.813     [ -0.364,  0.464]
beta[1]        0.9300      0.122      7.635  2.255e-14     [  0.691,  1.169]
============================================================================

Covariance estimator: robust
"""

#### GarchModel Class : Predict_volatility Method

In [9]:
#Generate prediction from 'model_netflix'
prediction = model_netflix.predict_volatility(horizon=5)
#Is prediction a dictionary?
assert isinstance(prediction,dict)
#Are keys correct data type?
assert all (isinstance(k,str) for k in prediction.keys())
#Are values correct data type?
assert all(isinstance(v,float) for v in prediction.values())
prediction

{'2025-05-26T00:00:00': 0.023,
 '2025-05-27T00:00:00': 0.024,
 '2025-05-28T00:00:00': 0.024,
 '2025-05-29T00:00:00': 0.024,
 '2025-05-30T00:00:00': 0.024}

#### GarchModel Class : Dump Method

In [10]:
#Save 'model_netflix' model assign file name
filename = model_netflix.dump()
#Is filename a string?
assert isinstance(filename,str)
#Does file name include ticker symbol?
assert model_netflix.ticker in filename
#Does file exists
assert os.path.exists(filename)
filename

'models\\2025-05-24T12-40-13.015974_NFLX.pkl'

#### GarchModel Class : Load Method


In [11]:
model_netflix = GarchModel(ticker="NFLX",repo=repo,use_new_data=False)
#Check that  new 'model_netflix_test' doesn't have model attached
assert not hasattr(model_netflix,"model")
#Load model
model_netflix.load()
#Does 'model_netflix_test' have model attached
assert hasattr(model_netflix,"model")
model_netflix.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                5397.07
Distribution:                  Normal   AIC:                          -10786.1
Method:            Maximum Likelihood   BIC:                          -10762.8
                                        No. Observations:                 2500
Date:                Sat, May 24 2025   Df Residuals:                     2499
Time:                        12:40:05   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         2.0727e-03  1.786e-03      1.161      0.246 [-1.427e-03,5.573e-03]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      2.0831e-05  7.320e-06      2.846  4.430e-03 [6.484e-06,3.518e-05]
alpha[1]       0.0500      0.211      0.237      0.813     [ -0.364,  0.464]
beta[1]        0.9300      0.122      7.635  2.255e-14     [  0.691,  1.169]
============================================================================

Covariance estimator: robust
"""

## 🌐 Creating a Simple Application Endpoint

### 🔹 `/hello` Path
We've created a `/hello` path for our application that returns a simple greeting when it receives a **GET** request.

### ✅ Test the GET Request
Now that the path is set up, let's perform a **GET** request to `/hello` to check if it works correctly.


In [12]:
url = "http://localhost:8008/hello"
response = requests.get(url=url)

print("response code :",response.status_code)
response.json()

response code : 200


{'message': 'Hello world!'}

## ⚙️ `/fit` Path

### 🔹 Purpose
Our first path allows the user to **fit a model to stock data** by making a **POST** request to the server.

- Users can choose to use **new data from AlphaVantage** or **existing data** already stored in our database.
- Upon making a request, the user will receive a **response** indicating whether the operation was **successful** or if there was an **error**.

---

## 🧪 `/fit` Path: Data Classes & Validation

With our **Pydantic data classes** defined, we can test how they:

- **Ensure users provide valid input**.
- **Guarantee the application returns correctly structured output**.

This validation ensures robust and predictable interactions with the `/fit` endpoint.


In [14]:
from main import FitIn,FitOut
#Instantiate "FitIn".play with parameters

In [15]:
fi = FitIn(
    ticker = "NTFL",
    use_new_data=True,
    n_observations=2500,
    p=1,
    q=1
)
print(fi)

ticker='NTFL' use_new_data=True n_observations=2500 p=1 q=1


In [16]:
#Instantiate "FitOut".Play with parameters
fo = FitOut(
    ticker = "NTFL",
    use_new_data=True,
    n_observations=2500,
    p=1,
    q=1,
    success=True,
    message="model is ready to rock"
    
)
print(fo)

ticker='NTFL' use_new_data=True n_observations=2500 p=1 q=1 success=True message='model is ready to rock'


### Build Model function

In [17]:
from main import build_model
#Intantiate "GarchModel" with function
model_netflix =build_model(ticker="NTFL",use_new_data=False)

In [18]:
#Is 'SQLRepository' attached to model_netflix?
assert isinstance(model_netflix.repo,SQLRepository)
#Is SQLite database attached to SQRepository
assert isinstance(model_netflix.repo.connection,sqlite3.Connection)
#Is ticker atribute correct?
assert model_netflix.ticker == "NTFL"
#Is 'use_new_data' attribute correct?
assert not model_netflix.use_new_data
model_netflix

#### Post Request to "Fit" path

In [19]:
import requests
response = requests.post("http://localhost:8008/fit",
                         json={"ticker":"NFLX",
                               "use_new_data":False,
                               "n_observations":2500,
                               "p":1,
                               "q":1}
                        )
response.json()

{'ticker': 'NFLX',
 'use_new_data': False,
 'n_observations': 2500,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "Trained and saved to 'models\\2025-05-24T12-41-10.508477_NFLX.pkl'."}

Boom! Now we can train models using the API we've created.Up next a path to make prediction

#### "Predict" path : Data classes

In [23]:
from main import PredictIn,PredictOut
pi = PredictIn(ticker="NTFL",n_days=5)
print(pi)

ticker='NTFL' n_days=5


In [24]:
po = PredictOut(
    ticker = "NTFL",
    n_days=5,
    success=True,
    forecast={},
    message="success"
)
print(po)

ticker='NTFL' n_days=5 success=True forecast={} message='success'


### "Predict" path : Build path

In [26]:
import requests
response = requests.post("http://localhost:8008/predict",
                        json={"ticker": "NFLX",
                              "n_days": 5}
                       )
response.json()

{'ticker': 'NFLX',
 'n_days': 5,
 'success': True,
 'forecast': {'2025-05-26T00:00:00': 0.023,
  '2025-05-27T00:00:00': 0.024,
  '2025-05-28T00:00:00': 0.024,
  '2025-05-29T00:00:00': 0.024,
  '2025-05-30T00:00:00': 0.024},
 'message': 'Forecast generated for NFLX over 5 days.'}

In [29]:
#Prediction Test
response = requests.post("http://localhost:8008/predict", json={"ticker":"NFLX","n_days":5})
assert response.status_code == 200
assert response.json()["success"]
assert len(response.json()["forecast"]) == 5
print("POST /predict passed")

POST /predict passed
